<a href="https://colab.research.google.com/github/tanyaachaturvedi/Sparse_RBM/blob/main/Sparse_RBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [33]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tanyaachaturvedi","key":"b3c75bbea56ad31f1033d54fa3fc6816"}'}

In [34]:
!mv "kaggle.json" kaggle.json  # rename it if needed
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

mv: 'kaggle.json' and 'kaggle.json' are the same file


In [35]:
# Download dataset
!kaggle datasets download -d crawford/gene-expression --force

# Unzip
import zipfile
with zipfile.ZipFile("gene-expression.zip", "r") as zip_ref:
    zip_ref.extractall("gene_expression_data")

Dataset URL: https://www.kaggle.com/datasets/crawford/gene-expression
License(s): CC0-1.0
  0% 0.00/1.41M [00:00<?, ?B/s]
100% 1.41M/1.41M [00:00<00:00, 486MB/s]


In [36]:
!ls gene_expression_data

actual.csv  data_set_ALL_AML_independent.csv  data_set_ALL_AML_train.csv


In [37]:
import pandas as pd

# Load all CSVs
actual_df = pd.read_csv("gene_expression_data/actual.csv")
train_df = pd.read_csv("gene_expression_data/data_set_ALL_AML_train.csv")
test_df = pd.read_csv("gene_expression_data/data_set_ALL_AML_independent.csv")

# Quick preview
print("Actual:")
print(actual_df.head())
print("\nTrain shape:", train_df.shape)
print("Test shape:", test_df.shape)


Actual:
   patient cancer
0        1    ALL
1        2    ALL
2        3    ALL
3        4    ALL
4        5    ALL

Train shape: (7129, 78)
Test shape: (7129, 70)


In [38]:
# Combine train and test
full_df = pd.concat([train_df, test_df], ignore_index=True)

# Merge labels into full_df
full_df["cancer"] = actual_df["cancer"]

# Drop non-feature columns if any
X = full_df.drop(columns=["cancer"])
y = full_df["cancer"]


In [39]:
# Set gene names as index and transpose to get samples as rows
full_df.set_index(full_df.columns[0], inplace=True)
X = full_df.drop(columns=["cancer"]).T  # transpose and drop label column
X = X.apply(pd.to_numeric, errors='coerce')  # ensure numeric

# Re-attach the labels
y = actual_df["cancer"].values

# Now normalize
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Standardizing the dataset

In [40]:
from sklearn.preprocessing import StandardScaler

In [41]:
scaler = StandardScaler()

In [42]:
import pandas as pd

# Load files
actual_df = pd.read_csv("gene_expression_data/actual.csv")
train_df = pd.read_csv("gene_expression_data/data_set_ALL_AML_train.csv")
test_df = pd.read_csv("gene_expression_data/data_set_ALL_AML_independent.csv")

# Combine expression data
expression_df = pd.concat([train_df, test_df], axis=1)


In [43]:
# First row in expression_df is gene names — set it as index
expression_df = expression_df.set_index(expression_df.columns[0])

# Transpose so that each row = 1 patient, each column = 1 gene
X = expression_df.T

# Ensure all values are numeric
X = X.apply(pd.to_numeric, errors='coerce')

In [44]:
from sklearn.preprocessing import LabelEncoder

# Get cancer types for each patient
y = actual_df['cancer']
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Normalizing

In [45]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [46]:
print("Features shape:", X_scaled.shape)
print("Labels shape:", y_encoded.shape)


Features shape: (146, 7129)
Labels shape: (72,)


In [47]:
print(X.shape)


(146, 7129)


# Applying PCA

In [48]:
from sklearn.impute import SimpleImputer

# Fill NaNs with column (gene) means
imputer = SimpleImputer(strategy='mean')
X_scaled_imputed = imputer.fit_transform(X_scaled)


In [49]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_scaled_imputed)


In [50]:
from sklearn.impute import SimpleImputer


In [51]:
imputer = SimpleImputer(strategy='mean')
X_scaled_imputed = imputer.fit_transform(X_scaled)


In [52]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_scaled_imputed)


In [53]:
explained = np.sum(pca.explained_variance_ratio_)
print(f"Total variance explained by 50 components: {explained:.2%}")


Total variance explained by 50 components: 89.58%


# Implementing Sparse RBM

In [57]:
import numpy as np
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder



In [62]:
import pandas as pd

df = pd.read_csv("gene_expression_data/data_set_ALL_AML_train.csv", index_col=0)


In [65]:
df = pd.read_csv("gene_expression_data/data_set_ALL_AML_train.csv", index_col=0)
print(df.head(10))
print(df.columns)


                                     Gene Accession Number    1 call    2  \
Gene Description                                                            
AFFX-BioB-5_at (endogenous control)         AFFX-BioB-5_at -214    A -139   
AFFX-BioB-M_at (endogenous control)         AFFX-BioB-M_at -153    A  -73   
AFFX-BioB-3_at (endogenous control)         AFFX-BioB-3_at  -58    A   -1   
AFFX-BioC-5_at (endogenous control)         AFFX-BioC-5_at   88    A  283   
AFFX-BioC-3_at (endogenous control)         AFFX-BioC-3_at -295    A -264   
AFFX-BioDn-5_at (endogenous control)       AFFX-BioDn-5_at -558    A -400   
AFFX-BioDn-3_at (endogenous control)       AFFX-BioDn-3_at  199    A -330   
AFFX-CreX-5_at (endogenous control)         AFFX-CreX-5_at -176    A -168   
AFFX-CreX-3_at (endogenous control)         AFFX-CreX-3_at  252    A  101   
AFFX-BioB-5_st (endogenous control)         AFFX-BioB-5_st  206    A   74   

                                     call.1    3 call.2    4 call.3    5  .

In [70]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the data
df = pd.read_csv("gene_expression_data/data_set_ALL_AML_train.csv", index_col=0)

# Drop metadata columns — keep only numeric expression columns
expression_cols = [col for col in df.columns if not col.startswith("call")]
X = df[expression_cols]

# Transpose so samples are rows and genes are features
X = X.T

# Convert to float and handle errors
X = X.apply(pd.to_numeric, errors='coerce')

# Drop any samples or genes with missing values
X = X.dropna(axis=0)  # drop rows (samples) with NaN
X = X.dropna(axis=1)  # drop columns (genes) with NaN

# Normalize the expression data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Final shape after cleaning:", X_scaled.shape)



Final shape after cleaning: (38, 7129)


In [73]:
# Load the labels from dataset_description.csv
labels_df = pd.read_csv("gene_expression_data/actual.csv")

# ONLY keep the first 38 labels to match X
labels_df = labels_df.iloc[:38]

# Get the target column
y = labels_df["cancer"].values  # This should now have shape (38,)

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Now split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print("Shapes after split:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)



Shapes after split:
X_train: (30, 7129)
X_test: (8, 7129)
y_train: (30,)
y_test: (8,)


In [74]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [75]:
from sklearn.neural_network import BernoulliRBM
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Initialize RBM and classifier
rbm = BernoulliRBM(n_components=100, learning_rate=0.01, batch_size=10, n_iter=20, verbose=True)
logistic = LogisticRegression(max_iter=1000)

# Pipeline
rbm_pipeline = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])


In [76]:
rbm_pipeline.fit(X_train_scaled, y_train)


[BernoulliRBM] Iteration 1, pseudo-likelihood = -4699.27, time = 0.04s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -4561.65, time = 0.04s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -4380.44, time = 0.04s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -4791.13, time = 0.04s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -4398.46, time = 0.14s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -4893.85, time = 0.10s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -4236.41, time = 0.08s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -4294.96, time = 0.92s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -4845.12, time = 0.35s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -4479.85, time = 0.14s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -4131.83, time = 0.04s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -4393.45, time = 0.04s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -3967.84, time = 0.04s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -4520.47, time = 0.04s
[

Pipeline(steps=[('rbm',
                 BernoulliRBM(learning_rate=0.01, n_components=100, n_iter=20,
                              verbose=True)),
                ('logistic', LogisticRegression(max_iter=1000))])

In [77]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = rbm_pipeline.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 0.5

Classification Report:
              precision    recall  f1-score   support

         ALL       0.50      1.00      0.67         4
         AML       0.00      0.00      0.00         4

    accuracy                           0.50         8
   macro avg       0.25      0.50      0.33         8
weighted avg       0.25      0.50      0.33         8



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [78]:
# Weights learned by RBM
weights = rbm.components_

# Find top contributing genes for each hidden unit
import numpy as np

top_genes_idx = np.argsort(np.abs(weights), axis=1)[:, -10:]  # Top 10 for each hidden unit


In [79]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Normalize features to [0, 1]
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define RBM + Classifier pipeline
rbm = BernoulliRBM(n_components=100, learning_rate=0.01, batch_size=10, n_iter=20, verbose=True)
logistic = LogisticRegression(max_iter=1000)
pipeline = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])

# Train
pipeline.fit(X_train_scaled, y_train)

# Evaluate
y_pred = pipeline.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Optional: Gene selection via top RBM weights
weights = rbm.components_  # Shape: [n_components, n_features]
top_features = np.argsort(np.abs(weights), axis=1)[:, -10:]  # Top 10 genes per component

print("\nTop genes per RBM hidden unit (as feature indices):")
print(top_features)


[BernoulliRBM] Iteration 1, pseudo-likelihood = -4718.19, time = 0.04s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -4628.52, time = 0.06s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -4206.38, time = 0.13s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -4500.53, time = 0.09s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -4435.73, time = 0.08s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -4731.73, time = 0.12s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -4679.97, time = 0.10s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -4214.16, time = 0.07s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -4730.87, time = 0.08s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -4692.29, time = 0.12s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -4559.22, time = 0.07s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -4754.72, time = 0.24s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -4211.95, time = 0.15s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -4297.02, time = 0.08s
[

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [80]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [81]:
rbm = BernoulliRBM(n_components=50, learning_rate=0.01, n_iter=100, random_state=42)


In [82]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)


In [83]:
print("Train label counts:", pd.Series(y_train).value_counts())
print("Test label counts:", pd.Series(y_test).value_counts())


Train label counts: 0    23
1     7
Name: count, dtype: int64
Test label counts: 1    4
0    4
Name: count, dtype: int64


In [86]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Scale input data to [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=42)

# Train RBM
rbm = BernoulliRBM(n_components=50, learning_rate=0.01, n_iter=100, random_state=42)
rbm.fit(X_train)

# Transform data with RBM
X_train_rbm = rbm.transform(X_train)
X_test_rbm = rbm.transform(X_test)

# Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_rbm, y_train)

# Predict and evaluate
y_pred = classifier.predict(X_test_rbm)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8333333333333334
              precision    recall  f1-score   support

         ALL       0.82      1.00      0.90         9
         AML       1.00      0.33      0.50         3

    accuracy                           0.83        12
   macro avg       0.91      0.67      0.70        12
weighted avg       0.86      0.83      0.80        12



In [87]:
classifier = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)


In [88]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_rbm, y_train)


In [89]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'class_weight': ['balanced']
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3)
grid.fit(X_train_rbm, y_train)
print("Best params:", grid.best_params_)


Best params: {'class_weight': 'balanced', 'max_depth': None, 'n_estimators': 100}


In [90]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', class_weight='balanced', probability=True)
svm.fit(X_train_rbm, y_train)


SVC(class_weight='balanced', probability=True)

In [91]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(classifier, X_train_rbm, y_train, cv=5)
print("Cross-val accuracy:", scores.mean())


Cross-val accuracy: 0.7
